# Setup

In [ ]:
#Imports 

import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.fftpack
from scipy.fft import fft, ifft
from scipy import stats
import math
import librosa
import IPython.display as ipd
import pandas as pd
from scipy.interpolate import interp1d
from scipy.io.wavfile import write
from scipy import interpolate
from scipy.optimize import minimize
from scipy.stats import iqr
import music21

In [ ]:
# Constants for reading .times or .ideal files

SECONDS_PER_AT_FRAME = 256/8000

fft_size=1024


hop_size=int(fft_size/4)
sr_ensemble=44100


fft_lim =513 #largest bin size for cutoff

#X seconds/sample * 512 samples/STFT frame

def seconds_to_stft_frames(seconds):
    samples = seconds * sr_ensemble
    return  math.floor((samples)/hop_size)


def stft_frames_to_seconds(stft_frames):
    samples = fft_size + (stft_frames-1)*hop_size
    return samples/sr_ensemble
    
    


half_step = 1/12 #multiply this by change to get 

# Load Files

### Load files describing original timepoints and unaltered audio



In [ ]:
path_to_ensemble_times="../../orchestra_part_times/mozart_serenade_eflat." #partial path for parsing

path_to_ensemble_audio="../../orchestra_part_audio/mozart_serenade_eflat." #partial path for parsing

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
other_instrument_dct = {
                 "oboe_1":[ 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"],
                 "oboe_2":
                  ["oboe_1",   
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"],
    
                  "clarinet_1": 
                ["oboe_1", 
                  "oboe_2", 
                    "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"], 
    
                  "clarinet_2":
                 ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"],
    
                  "bassoon_1":
                 ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"],
    
    
                  "bassoon_2":
                 ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "horn_in_e_1", 
                  "horn_in_e_2"],
    
                  "horn_in_e_1": 
                 ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_2"], 
                  "horn_in_e_2":
                 ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1"]
}
#number of parts
num_voices=8
#sanity check
num_voices == len(instrument_lst)



Load Original Parts Audio

In [ ]:
#Load each file into a list
ensemble_audio_lst = []
sr_dct={}
for instr in instrument_lst:
    print(instr)
    samples, sr = librosa.load(path_to_ensemble_audio+instr+".wav", sr=None)
    if sr==sr_ensemble:
        samples_48k=samples
    else:
        samples_48k = librosa.resample(samples, sr, sr_ensemble, 'kaiser_fast')
    ensemble_audio_lst.append(samples_48k)


#should be (num_instruments, length_samples)
ensemble_audio_lst

In [ ]:
#
part_stfts = [ ]

for i in  range(0, num_voices):
    part_stfts.append(
        librosa.stft(ensemble_audio_lst[i], n_fft=fft_size, hop_length=hop_size, center=True))


## Create list of STFTs for each part 

In [ ]:
part_stfts[0].shape

### Extract data from each times file and tunign file

In [ ]:
path_to_ensemble_tun = '../../tuning/mozart_serenade_eflat.'

In [ ]:



df_dct={} #list with dataframe corresponding to each one 



for instr in instrument_lst:
    

    '''
    Read in .times file as dataframe
    '''
    times_df = pd.read_csv(path_to_ensemble_times+instr+".times", 
                              header=None,
                              delim_whitespace=True,
                              names=['measure', 'AT', 'marked'])


    '''
    Read in tuning file
    '''
    tun_df = pd.read_csv(path_to_ensemble_tun+instr+".tun", 
                              header=None,
                              delim_whitespace=True,
                              names=['measure','shift (half step)'])

    #parse out the word 'solo' so can join with times df
    remove_solo = lambda x: x[5:len(x)]

    tun_df['measure']=list(map(remove_solo, tun_df['measure']))
    
    '''
    get absolute pitch shift by multiplying by half step constant 
    '''
    get_abs_tun = lambda x: 2 ** (-1*x*half_step)#Guessing positive value -> flatter
    
    tun_df['tune (abs)'] = list(map(get_abs_tun, tun_df['shift (half step)']))

    '''
    Don't care about marked
    '''
    times_df = times_df[['measure', 'AT']]

    '''
    Remove extra rows
    '''

    times_df=times_df[4:len(times_df)-1].reset_index(drop=True)



    '''
    Join
    '''
    times_df= times_df.set_index('measure').join(tun_df.set_index('measure')).reset_index()
    
    '''
    use mapping to cast measure
    '''
    evalfun = lambda x: eval(x)
    evaluatedMeasure=list(map(evalfun, times_df['measure']))

    times_df['eval_measure']=evaluatedMeasure        


    '''
    use mapping to AT to float
    '''
    try:
        evaluatedsec=list(map(evalfun, times_df['AT']))
        times_df['AT']=evaluatedsec
    except:
        print("casting done automatically by pandas")

        
    '''
    Pad last element AT (so it doesn't cut off)
    '''
    times_df['AT'].iloc[-1]= times_df.iloc[-1]['AT']+15

    '''
    Convert AT to seconds
    '''
    times_df['Seconds']=times_df['AT']*SECONDS_PER_AT_FRAME 

    '''
    Get STFT frame number at each timepoint
    '''

    evaluatedstft=list(map(seconds_to_stft_frames, times_df['Seconds']))
      

    times_df['STFT frames']=evaluatedstft

    '''
    Add to dictionary
    '''
    df_dct[instr]=times_df




# Remove Ghost Notes

In [ ]:
path_to_mid ="../../mozart_serenade_eflat/mozart_serenade_eflat.mid"


mid_instrument_lst= ['oboe_1',
 'oboe_2',
 'clarinet_1',
 'clarinet_2',
                      'horn_in_e_1',
 'horn_in_e_2',
 'bassoon_1',
 'bassoon_2',
]

In [ ]:

'''
For a given part gets music21 notes/chords, measure metric used in Cadenza, 
    and each note's offset in quarter notes
Params:
part: music21.stream.Part, part to parse

Returns: 

list of music21 notes/chords, measure metric used in Cadenza, 
    and each note's offset in quarter notes

Returns :
offsets, eval_measures, notes- lists of.....
    'notes': music21 note/chords
    'evaluated_measure' : used in Cadenza
    'beat position' : note offset in quarter note beats 
'''
def extract_info_from_part(part):
    offsets=[] #offset in beats
    eval_measures=[] #same form as cadenza
    notes=[] #music21 note or chord
    cur_measure_offset=0

    for measure in part.getElementsByClass(music21.stream.Measure):
        
        if measure.timeSignature != None:
            cur_ts = measure.timeSignature
            beatVal = 2 **(2-math.log(cur_ts.denominator)/math.log(2))
            num_beats = cur_ts.numerator
            measure_len = beatVal * num_beats

        
        
        for note in list(measure.recurse().notes):
            offsets.append(cur_measure_offset + note.offset)
            eval_measures.append(measure.number + note.offset/4)
            notes.append(note)
        cur_measure_offset += measure_len
    
    return offsets, eval_measures, notes

b = music21.converter.parse(path_to_mid)
b.show('text')


parts = b.getElementsByClass(music21.stream.Part)
for i in range(8):
    print('ON INDEX', i)
    parts[i][0].show('text')
    


In [ ]:
for i in range(len(parts)):
    part_info= extract_info_from_part(parts[i])
    keep_times = [float(time) for time in part_info[1]]
    instr_df = df_dct[mid_instrument_lst[i]]
    #append last time 
    keep_times.append(instr_df['eval_measure'].iloc[-1])
    locations = instr_df[instr_df['eval_measure'].isin(keep_times)].index
    new_df = instr_df.loc[locations].reset_index(drop=True)
    print(part_info[2])
    print(new_df)
    df_dct[mid_instrument_lst[i]]=new_df

## Audio Stretching functions

In [ ]:
def pitch_shift_stretch1(audio_samples_full, start_sample, 
                         end_sample, prev_phase, sr, 
                         shift_constant, 
                         desired_frame_num, 
                         first_note, fft_size, hop_size, window, fft_height):

    
    '''
    We want audio from start of current frame to fft_size past ending point to calculate phases.
    Will only use last frame to calculate phase advance - will not save modulus
    
      note start 
       |----
         ----
          ---- 
           ---- note end
            ----|
             ---|-
              --|--
               -|---
                |----
    '''
    
    audio_samples = audio_samples_full[start_sample:end_sample+fft_size*2] 
    #** fft_size *2 gives some buffer audio for ptich shifting (want at least fft_size)
    #print("shape of audio_samples is ",audio_samples.shape)
    
    '''
    Part I: Perform Pitch shift on all audio 
    '''
    #print("fft_size ", fft_size)
    #print("hop size", hop_size)
    N_note=len(audio_samples)
    total_time = N_note/sr_ensemble
    
    #original timepoints
    x = np.linspace(0, total_time, num=N_note, endpoint=True)

    #interpolation function 
    sample_interpolation = interp1d(x, audio_samples, kind='linear')

    #New timepoints
    xshift = np.linspace(0, total_time, num=int(N_note/shift_constant), endpoint=True) 
    #apply interpolation function to new timepoints
    yshift = sample_interpolation(xshift)

    '''
    Part II: Time stretch
    
    We want new audio to occupy desired_frame_num amount of frames 
    Generate  desired_frames+1 timepoints from [start_sample to end_sample]
    For each timepoint except first, phase differential is difference between phase of current timepoint fft 
                and phase of current timepoint - (hop size?) fft 
                ** test what happens when this is previous index fft differential 
    
    
    If we are at beginning, initial phase is preserved as first phase difference
    Else, we are plugging in the last calculated phase difference of previous note
    '''
    
    #timpoints  
    N_shifted = int((end_sample-start_sample)/shift_constant) #want start of first note to start of next_note 
    timepoints = np.linspace(0, N_shifted, num=desired_frame_num+1, endpoint=True).round(0).astype(int)

    #iterate through creating frames - making 1 extra 
    mod = np.zeros((int(fft_size/2)+1, desired_frame_num+1))
    diff_phase = np.zeros(mod.shape)

    for i in range(desired_frame_num+1): 

        #Get position from timepoints 
        sample_index = timepoints[i]
        
        #Create fft starting from sample index 
        chunk1start=sample_index
        chunk1end=chunk1start+fft_size
        chunk1 = yshift[chunk1start:chunk1end] *window
        fft1= fft(chunk1)
        
        #Mod
        mod[:,i]=np.abs(fft1[0:fft_height ])

        #Phase 
        
        if i==0:
            
            #if it's the first note, keep its current phase 
            if first_note == True:
                frame_phase = np.angle(fft1)
                diff_phase[:,0] = frame_phase[0:fft_height ]
            #otherwise sub in last phase advance 
            else:
                diff_phase[:,0:1] = prev_phase
        else:
            #phase differential is calculated based on fft starting at cur_position - hop_size 
            chunk2start = sample_index-hop_size
                
            chunk2end = chunk2start+fft_size
            #If we're going sharp, first value will be less than 0
            if chunk2start <0:
                chunk2=np.zeros(fft_size)
                chunk2[0-chunk2start:]=yshift[0:chunk2end]
                chunk2=chunk2*window
            #normal case 
            else:
                chunk2 = yshift[chunk2start : chunk2end]*window
            fft2 = fft(chunk2) 
            
            #take difference 
            frame_phase = np.angle(fft1)[0:fft_height ]-np.angle(fft2)[0:fft_height ] 
            diff_phase[:,i]=frame_phase

    #Remove last element, saving the phase advance
    
    joining_phase_advance = diff_phase[:, -1:]
    
    
    
    diff_phase = diff_phase [:, 0:-1]
    mod=mod[:, 0:-1]
    
    return mod, diff_phase, joining_phase_advance
    




In [ ]:
def vocode_to_new_frames(df, part_samples, new_end_time):
    #new end time and new end frame should be same for everyone (if not some can be cut )

     #constants
    new_end_frame = seconds_to_stft_frames(new_end_time)
    window=scipy.signal.windows.hann(fft_size)
    fft_height = int(fft_size/2+1) #height of fft (# bins)


    #fillable arrays
    vocoded_mod= np.zeros((fft_height, new_end_frame))
    vocoded_diff_phase= np.zeros((fft_height, new_end_frame))

    start_og_sample = int(df.iloc[0]['Seconds']*sr_ensemble)#Audio frame in original playing begins
    start_new_frame = df.iloc[0]['New STFT frames']

    num_measures = len(df)
    for i in range(0, num_measures-1):

        end_og_sample = int(df.iloc[i+1]['Seconds']*sr_ensemble)
        end_new_frame = df.iloc[i+1]['New STFT frames']

        #print(start_og_sample, "start og sample")
        #print(end_og_sample, "end og sample")


        pitch_shift = df.iloc[i]['tune (abs)']
        #print("************\n"+df.iloc[i]['measure'])
        #print("shifting to pitch ", pitch_shift)
        #Calculate nessesary time stretch
        if end_og_sample < start_og_sample:
            print("END OG FRAME IS LESS THAN START OG FRAME")
            print(start_og_sample)
            print(end_og_sample)

        new_frame_dif = end_new_frame - start_new_frame


        '''
        Changes

        Hypothesis from test 2: just STFT process resutls in 1 more? frame than expected
        Resutl: 
        '''        


        #if we are on first chord, preserve initial phase 
        if i==0:


            mod, diff_phase, end_phase = pitch_shift_stretch1(audio_samples_full=part_samples,
                                                              start_sample=start_og_sample, end_sample=end_og_sample, 
                                                              prev_phase=None, sr=sr_ensemble, shift_constant=pitch_shift, 
                                                              desired_frame_num = new_frame_dif, first_note=True, 
                                                              fft_size=fft_size, hop_size=hop_size, 
                                                              window=window, fft_height=fft_height)

        else:


            mod, diff_phase, end_phase = pitch_shift_stretch1(audio_samples_full=part_samples,
                                                              start_sample=start_og_sample, end_sample=end_og_sample, 
                                                              prev_phase=end_phase, sr=sr_ensemble, shift_constant=pitch_shift, 
                                                              desired_frame_num = new_frame_dif, first_note=False, 
                                                              fft_size=fft_size, hop_size=hop_size, 
                                                              window=window, fft_height=fft_height)


        correct_len = end_new_frame-start_new_frame
        #print("length should be ", correct_len)
        #print("new chunk shape", mod.shape[1])


        #Put vocoded chunk in final STFT

        vocoded_mod[:,start_new_frame:end_new_frame]=mod
        vocoded_diff_phase[:,start_new_frame:end_new_frame]=diff_phase
        
        #increment samples

        start_og_sample = end_og_sample
        start_new_frame = end_new_frame



    
    return vocoded_mod, vocoded_diff_phase


In [ ]:

def create_final_parts(ensemble_audio_lst,df_dct,instrument_lst, end_pad ): #end pad is in ms
    adj_part_lst = []
    
    #pick longest for new_end_time
    new_end_time = 0
    for voice in instrument_lst:
        
        cand_end_time = df_dct[voice].iloc[len(df_dct[voice])-1]['new_times']+end_pad
        
        if cand_end_time> new_end_time:
            new_end_time = cand_end_time
    for i in range(0, len(instrument_lst)):
        print("***************************************************************************************************")
        print("Instrument: ",instrument_lst[i] )
        #get correct stft and dataframe to describe instrument
        part_samples=ensemble_audio_lst[i]
        df = df_dct[instrument_lst[i]]
        #vocode part so it fits the new timing scheme
        vocoded_mod, vocoded_diff_phase = vocode_to_new_frames(df, part_samples, new_end_time)
        cum_resampled_phase = np.cumsum(vocoded_diff_phase, axis=1)

        #combine modulus and argument to make final stft
        cum_stft = vocoded_mod*np.exp(1j*cum_resampled_phase) 

        #add to list
        adj_part_lst.append(cum_stft)
    return adj_part_lst    


In [ ]:
def sum_parts(adj_part_lst):
        #sum parts 
    sum_vocoded_stft= np.zeros((int(fft_size/2+1), adj_part_lst[0].shape[1]), dtype='complex')
    for i in range(0, len(instrument_lst)):
        #add waves
        sum_vocoded_stft += adj_part_lst[i]*nonvar_mix[i]
        
    #convert to audio sphere
    adjusted_by_measure = librosa.istft(sum_vocoded_stft,  hop_length=hop_size)
    return adjusted_by_measure

In [ ]:
def gen_mix_parts(adj_part_lst, nonvar_mix, folder_path = None):
    
    if folder_path==None:
        print("Must supply path of folder to write parts. Create first")
        return 0
    print("writing to "+folder_path)    
    #list of parts   
    part_samples_lst = []
    
    for i in range(0, len(instrument_lst)):
        #convert into sample space
        samples_float = librosa.istft(adj_part_lst[i]*nonvar_mix[i],hop_length=hop_size)
        samples_int = (2**16*samples_float).astype(np.int16)
        part_samples_lst.append(samples_int)
        print("writing part for ", instrument_lst[i])
        write(folder_path+"/"+instrument_lst[i]+".wav", sr_ensemble, samples_int)
        

    return part_samples_lst

In [ ]:
#input df version with NaN, so we can fill with w_i
#w_i must be less than 1/n_p
def make_weight_arr(comb_df,instrument_lst, w_i ):
    
    weight_arr = comb_df[instrument_lst].fillna(w_i).to_numpy()
    weight_arr[weight_arr>w_i ] = 0 #value to be filled
    sums = 1- np.sum(weight_arr, axis=1)
    for i in range(len(weight_arr)):
        row = weight_arr[i]
        num_actual_voices = num_voices-np.count_nonzero(row)
        weight = sums[i]/num_actual_voices
        #print(weight)
        row[row ==0]=weight
        weight_arr[i]=row
        print(weight_arr[i])
    return weight_arr

## Constants for nonvariable mix
- for final version do manual adjustment

In [ ]:
nonvar_mix = np.array([[0.1222869 ],
       [0.26452896* .75],
       [0.20447949*2],
       [0.63013774],
       [0.2666939*.75 ],
       [1.8622239 ],
       [0.322688  ],
       [0.3939167* 1.2 ]], dtype=np.float32)

# Create df with all eval_measure and STFT frames



In [ ]:
df_STFT_dct={}

for v in instrument_lst:
    df_STFT_dct[v]= df_dct[v][['eval_measure', 'STFT frames']]
    df_STFT_dct[v][v]=df_STFT_dct[v]['STFT frames']
    df_STFT_dct[v]=df_STFT_dct[v][['eval_measure', v]]

In [ ]:
comb_df = df_STFT_dct['oboe_1'].set_index('eval_measure')

for v in instrument_lst[1:]:
    comb_df = comb_df.join(
    df_STFT_dct[v].set_index('eval_measure'), 
    #on = 'eval_measure', 
    how='outer')

In [ ]:
comb_df = comb_df.reset_index()
comb_df

### Extrapolate missing values for mean calculation

- Want tempo in interpolated sections to accelerate from tempo_old to tempo_new. 
- Method 1: Avg tempo
    - Extrapolate forward from prev section and backward from next section. Average times. 
- Method 2: Weighted avg tempo
    - Closer to forward implies tempo closer to forward, closer to backward implies tempo closer to backward. Constants calculated based from measure


In [ ]:
#part measures is df_dct[v]['eval_measure']
#max_dist is maximum gap between adjacent notes (in measures)
'''
Aim: everything under 2 measures is interpolated linearly, otherwise tempo extrapolated 
'''
def find_blocks(part_measures, max_dist = 1):
    block_lst = []
    cur_block = []
    for i in range(len(part_measures)-1):

        cur_measure = part_measures[i]
        cur_block.append(i)
        #check distance from next block
        next_measure = part_measures[i+1]

        if next_measure - cur_measure > max_dist : 
            #single notes do not constitute block
            if len(cur_block)>1:
                block_lst.append(cur_block)
            cur_block = []
    if len(cur_block)>0:
        cur_block.append(len(part_measures)-1) #add last element
        block_lst.append(cur_block)
    return block_lst   

In [ ]:
def make_extrapolate_df(nan_df, df_dct, all_measures, method = "avg"):
    extrap_df = nan_df.copy()

    for v in instrument_lst:
        x = np.array(df_dct[v]['eval_measure'])
        print(x)
        y = np.array(df_dct[v]['STFT frames']) 
        blocks = find_blocks(x)
        
        print("BLOCKS\n", blocks)
        #store extrapolations
        forward_extrapolation_lst = []
        backward_extrapolation_lst =[]
        for i in range(len(blocks)):
            print("\n******************************\nOn block", i, "\n*******")
            block = blocks[i]
            '''
            1. Interpolate within blocks to get locations of local rests
            '''
            #fit function
            
            print("fitting: ", x[block], "to", y[block])
            f = interp1d(x[block], y[block] , fill_value = 'extrapolate')
            #figure out combined index locations 
            start_measure = x[block[0]]
            end_measure = x[block[-1]]
            print("starting on measure", start_measure, "ending on measure", end_measure)
            start_comb_idx = all_measures.index(start_measure)
            end_comb_idx = all_measures.index(end_measure)
            print('In full version starts on index', start_comb_idx, "and ends on index",  end_comb_idx)
            #perform interpolation within block 
            y1 = f(all_measures[start_comb_idx: end_comb_idx+1]) #including ending index for interpolation
            print("After interpolation:", all_measures[start_comb_idx: end_comb_idx+1],
                 "maps to ", y1)

            #assign
            extrap_df[v].loc[start_comb_idx: end_comb_idx] = y1
            print("currently, dataframe is \n", extrap_df[v])
            
            '''
            2. Perform forward extrapolation 
            '''
            print("\n&&&&\nExtrapolation")
            #start extrapolation at last value of current block
            forward_extrap_start = x[blocks[i][-1]]
            #end extrapolation on first value of next block or end of piece
            if i < len(blocks)-1:
                forward_extrap_end = x[blocks[i+1][0]]
            else:
                forward_extrap_end = all_measures[-1]
            #extrapolate vals
            
            forward_start_idx = all_measures.index(forward_extrap_start)
            forward_end_idx = all_measures.index(forward_extrap_end)
            #measures slice
            forward_x = all_measures[forward_start_idx: forward_end_idx+1 ] #include end
            forward_y = f(forward_x)
            print("In forward extrapolation: mapped", forward_x, "to", forward_y)
            #make start 0 to get differences 
            forward_y = forward_y - forward_y[0]
            
            #add x y pairs to dictionary
            forward_extrapolation_lst.append({'startval':forward_start_idx, 
                                              'endval':forward_end_idx,
                                              'y':forward_y})
            '''
            2. Perform backward extrapolation 
            '''
            #start at beginning of piece or end of last block
            if i > 0:
                backward_extrap_start = x[blocks[i-1][-1]]
            else:
                backward_extrap_start =all_measures[0]
            #end on first beat of this block
            backward_extrap_end =    x[blocks[i][0]]
            
            backward_start_idx = all_measures.index(backward_extrap_start)
            backward_end_idx = all_measures.index(backward_extrap_end)
            
            
            backward_x = all_measures[backward_start_idx: backward_end_idx+1] #include end
            backward_y = f(backward_x)
            print("In backward extrapolation: mapped", backward_x, "to", backward_y)
            backward_y = backward_y - backward_y[0]
            
            backward_extrapolation_lst.append({'startval':backward_start_idx,
                                               'endval':backward_end_idx , 
                                               'y':backward_y})
 
        print("\n\n@@@@@@@@@@@@@@@\n Part 3: filling in long NAN values")

        '''
        Decide final extrapolated values
        '''
        #Fill beginning and ending values
        beginning = backward_extrapolation_lst[0]  
        print('beginning dct', beginning)
        # TODO: this should move backward from last existing value
        extrap_df[v].loc[beginning['startval']: beginning['endval']] = (
            extrap_df[v].loc[beginning['endval']]- beginning['y'][-1])+ beginning['y']
        
        print(extrap_df[v].loc[beginning['endval']]- beginning['y'][-1])
        print(beginning['y'])
        backward_extrapolation_lst.pop(0)
        
        ending = forward_extrapolation_lst[-1] 
        #for ending values, add differences on last existing value
        extrap_df[v].loc[ending['startval']: ending['endval']] = ending['y'] +extrap_df[v].loc[ending['startval']]
        forward_extrapolation_lst.pop(-1)      
        
        print("dataframe\n", extrap_df[v])
        
        print("\n\n$$$$$$$$$$\nExtrapolating long rests and moving forward\n")
        
        
        print('backward extrap info\n', backward_extrapolation_lst[0:3])
        print('forward extrap info\n', forward_extrapolation_lst[0:3])
        #Rest of long rests:
        for i in range(len(backward_extrapolation_lst)):
            print("SHOULD BE SAME")
            print("starts", backward_extrapolation_lst[i]['startval'], 
                 forward_extrapolation_lst[i]['startval'])
            print("ends", backward_extrapolation_lst[i]['endval'], 
                 forward_extrapolation_lst[i]['endval'])
            
            forward_y = forward_extrapolation_lst[i]['y']
            backward_y = backward_extrapolation_lst[i]['y']
            start_idx = forward_extrapolation_lst[i]['startval']
            end_idx = forward_extrapolation_lst[i]['endval']
            start_frame = extrap_df[v].loc[start_idx]
            end_frame = extrap_df[v].loc[end_idx]
            print("start frame", start_frame)
            print("end_frame", end_frame)
            '''
            Option 1: average:  
            '''
            forward_y_arr = np.array(forward_y)
            print("FORWARD:\n", forward_y_arr)
            backward_y_arr = np.array(backward_y)
            print("Backward:\n", backward_y_arr)
            if method=='avg':
                print("forward_y", forward_y )
                print("backward_y",backward_y)
                y_extrap = (forward_y_arr + backward_y_arr ) /2
          
            #Option 2: smooth interpolation
          
            elif method=='smooth_interp':
                forward_y_weights = np.linspace( 1,0, len(forward_y_arr))
                backward_y_weights = 1-forward_y_weights
                print("forward_y_weights", forward_y_weights)
                print("backward_y_weights", backward_y_weights)
                print("sum weight forward", forward_y_weights * np.diff(forward_y_arr, prepend=0))
                print("sum_weight_backward", backward_y_weights*np.diff(backward_y_arr, prepend=0)  )              
                y_diffs = (forward_y_weights * np.diff(forward_y_arr, prepend=0)
                            + backward_y_weights*np.diff(backward_y_arr, prepend=0))
                y_extrap = np.cumsum(y_diffs)
                
                print("in method, extrapolated vals are\n", y_extrap)
            #add to last value
            y_extrap = y_extrap + start_frame
            
            
            '''
            Perform assignment of nan values and shifting of values after
            '''
       
            #assign extrapolation to NAN values
            print("y_extrap\n", y_extrap)
            print("putting values at ", extrap_df[v].loc[start_idx: end_idx])
            extrap_df[v].loc[start_idx: end_idx] = y_extrap
            print("df\n", extrap_df[v])
            #shift forward previous values
            shift = extrap_df[v].loc[end_idx]-end_frame
            print("next section should shift by ", shift)
            if i< len(backward_extrapolation_lst) - 1: 
                next_start_idx = forward_extrapolation_lst[i+1]['startval']
            else: 
                next_start_idx = len(all_measures)
                
            print("inserting into\n", extrap_df[v].loc[end_idx: next_start_idx])
            print("end idx", end_idx)
            print("next start idx", next_start_idx)
            extrap_df[v].loc[end_idx+1: next_start_idx] = extrap_df[v].loc[end_idx+1: next_start_idx]+shift
        extrap_df[v] = extrap_df[v]- extrap_df[v].loc[0]

    return extrap_df

In [ ]:
all_measures = list(comb_df['eval_measure'])

In [ ]:
filled_comb_df = make_extrapolate_df(comb_df, df_dct, all_measures, method = 'avg')
filled_comb_df

Check interpolation quality

In [ ]:
plt.figure()
plt.plot(filled_comb_df['eval_measure'], filled_comb_df['clarinet_2'], 'bo')



# Other utility functions

In [ ]:
def assign_new_stft(df_dct, optimized_df, instrument_lst):
    df_dct1 = df_dct.copy()
    for v in instrument_lst:

        frames_df=  optimized_df[['eval_measure']].copy()
        frames_df['New STFT frames']=optimized_df[v].astype(int) #was hardcoded oboe_1
        frames_df["new_times"] = stft_frames_to_seconds(frames_df['New STFT frames'])
        frames_df= frames_df.set_index('eval_measure')

        #join
        df_dct1[v]=df_dct[v].set_index('eval_measure').join(frames_df)
    return df_dct1

In [ ]:
def plot_tempos(x0, x_gd, del_m_series, start=0, stop=None, ylow=None, yhigh=None, show_rates=False):
    for p in range(n_p):

        d = np.diff(x0[:,p])


        e = np.diff(x_gd[:,p])


        plt.figure()
        plt.title(instrument_lst[p])
        plt.plot((d/del_m_series)[start:stop], label='old tempo')
        plt.plot((e/del_m_series)[start:stop], label = 'new ')
        if show_rates == True:
            print("Rate values", (e/del_m_series)[start:stop])
        plt.ylim([ylow, yhigh])
        plt.legend()
        plt.show()

In [ ]:
hand_parsed_leader1 = [['clarinet_1', (0,5)], 
                      ['oboe_1',(5,8)],
                      ['oboe_2', (8,9)],
                      ['horn_in_e_1', (9,13)],
                     [ 'bassoon_1',(13,14.5)],
                     [ 'horn_in_e_1',(14.5, 16.5)],
                     [ 'oboe_2', (16.5,17)],
                      ['horn_in_e_1',(17, 17.5)],
                      ['oboe_1',(17.5, 18)],
                      ['horn_in_e_1',(18, 18.5)],  
                      ['clarinet_2',(18.5, 19)],
                      ['horn_in_e_1',(19, 19.5)],
                      ['clarinet_1',(19.5, 20)],
                      ['bassoon_2',(20, 20.5)],
                      ['bassoon_1',(20.5, 21.5)],
                      ['horn_in_e_1',(21.5, 22.5)],
                      ['oboe_1',(22.5,24)],
                      ['bassoon_1',(24, 25)],
                      ['oboe_1',(25, 26.5)],
                      ['clarinet_1',(26.5, 30.5)],
                      ['oboe_1',(30.5,38)],
                      ['clarinet_1',(38, 39.5)],
                      ['oboe_1',(39.5, 43)], 
                      ['clarinet_1',(43,45.5)],
                      ['clarinet_2',(45.5, 46.5)],
                      ['oboe_1',(46.5, 48)],
                      ['horn_in_e_2',(48, 49)],
                      ['clarinet_2',(49, 50)],
                      ['bassoon_1',(50, 51)],
                      ['horn_in_e_2',(51, 52)],
                      ['clarinet_2',(52, 53)],
                      ['bassoon_1',(53, 54)],                      
                      ['horn_in_e_2',(54, 55)],
                      ['clarinet_2',(55, 56)],
                      ['bassoon_1',(56, 57)], 
                      ['clarinet_2',(57, 59)], 
                      ['oboe_1',(59, 60)], 
                      ['horn_in_e_2',(60, 60.5)],
                      ['clarinet_1',(60.5, 61)], 
                      ['horn_in_e_1',(61, 61.5)],
                      ['clarinet_2',(61.5, 62)],
                      ['horn_in_e_1',(62, 62.5)],
                      ['oboe_1',(62.5, 63)],
                      ['horn_in_e_1',(63, 63.5)],
                      ['oboe_2',(63.5, 64)],
                      ['bassoon_2',(64, 64.5)],
                      ['bassoon_1',(64.5, 65.5)],
                      ['clarinet_1',(65.5, 66.5)],
                      ['clarinet_2',(66.5, 67.5)],
                      ['oboe_2',(67.5, 69)],
                      ['clarinet_2',(69, 70)], 
                      ['clarinet_1',(70, 71)], 
                      ['horn_in_e_2',(71, 72)],
                      ['clarinet_1',(72, 74.5)],
                      ['oboe_1',(74.5, 78)],
                      ['clarinet_1',(78, 79.5)],
                      ['oboe_1',(79.5, 80)],
                      ['bassoon_1',(80, 82.5)],
                      ['clarinet_1',(82.5, 91)]]




for key in hand_parsed_leader1 :
    if key[0] not in instrument_lst:
        print(key, hand_parsed_leader[key])

# Follow the Leader

In [ ]:

def follow_leader_manual(hand_parsed_lst, filled_comb_df, instrument_lst):
    prev_val=0
    hand_df = filled_comb_df.copy()
    
    for i in range(len(hand_parsed_lst)):
        #current block
        leader = hand_parsed_lst[i]
        instr=leader[0]
        lims=leader[1]

        print('lims[0] is ', lims[0])

        cond = ((filled_comb_df['eval_measure']<=lims[1])&
                        (lims[0] <=filled_comb_df['eval_measure']) )

        raw_leader_times = (filled_comb_df[instr]).loc[cond]
        end_idx =  raw_leader_times.head(1).index[0]
        print('end_idx',end_idx)
        #calculate shift
        original_start = filled_comb_df[instr].iloc[end_idx]
        print('origianl start', original_start)

        beg = hand_df[instr].iloc[end_idx]
        move = beg - original_start
        print('beginning is ', beg)
        print("prev val is ", prev_val)
        print('move is ', move)

        
        
        

            
        print("prev val is ", prev_val)
        leader_times = raw_leader_times + move
        leader_tiled =np.tile(np.array(leader_times), (len(instrument_lst)+1,1)).T
        print(leader_times)
        
        
        #previous end 


        hand_df.loc[cond]= leader_tiled

        #find previous value 
    hand_df['eval_measure']= filled_comb_df['eval_measure']
    return hand_df    

In [ ]:
norm_constant = 60000/filled_comb_df[instrument_lst].iloc[-1]

In [ ]:
normalized_fc_df = filled_comb_df[instrument_lst]*norm_constant
normalized_fc_df['eval_measure']= filled_comb_df['eval_measure']

In [ ]:

times_manual = follow_leader_manual(hand_parsed_leader1, normalized_fc_df, instrument_lst)

In [ ]:
#pd.set_option('display.max_rows', None)

In [ ]:
final_stretch = assign_new_stft(df_dct, times_manual, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
write("whole_audio/mozart_follow_leader_hp1.wav", sr_ensemble, stretched_full_audio)

# Simulation

- everything starts at same tempo. 
- next timepoint is a * group_pred_position + b * self_desired_position + c* value interpolated by current tempo


##### Slow down phenomena


- Why? 
- If one instrument takes time then speeds up, other instruments will react to the slow down, slowing down the average tempo. When the original instruments speeds up again, they are influenced by the new slower average tempo and don't gain the time back
- This leads to negative feedback loop that makes everything go slower and slower


How to fix?
- Steady leaders/leader
    - "Accompaniment" instruments listen only to solid beat instrument, this way they are not swayed by soloists
    - "Solo" instruments listen to accomapniment instrumnets to stay generally on the beat
    
- Tempo correction term
    - Part of next step depends on a target tempo, preventing excessive slowdowns

$t_{j+1} = t_{i,j}+ w_{tog}G_{tog}(j) + w_{self}G_{self}(i,j) + w_{tempo}G_{tempo}(j)$, 





$G_{tog}(j)= (m_{j+1}-m_{j})\frac{1}{n_p}\sum^{n_p}_{i=1}\frac{t_{i, j}-t_{i,j-1}}{m_j-m_{j-1}}$

$G_{self}(i,j)= \frac{t^0_{i,j+1}-t^0_{i,j}}{t^0_{i,j}-t^0_{i, j-1}}(t_{i,j}-t_{i, j-1})$

$G_{tempo}(j)=(m_{j+1}-m_{j})R_{j}$


In [ ]:
#outputs normalized timing df where 1st beat is the same length used for simulations
def normalize_df(input_df, tempo):
    df = input_df.copy()
    #initialize
    m0 = df.loc[0]
    m1 = df.loc[1]
    diff0 = m1-m0
    #switch with frames_per_measure later
    first_beatlen = diff0['eval_measure']* tempo
    #What multiple is needed to normalize tempo for all parts
    tempo_multiple = diff0/first_beatlen
    #normalize tempo by matching length of first note
    df[instrument_lst]=(df / tempo_multiple)[instrument_lst]
    return df

## Simulation versions

Old version: does not allow for target tempo loss or variable listening

In [ ]:

def simulate_performance(input_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, 
                         tempo,catch_reverse=True):

    df = input_df.copy()
    #initialize
    m0 = df.loc[0]
    m1 = df.loc[1]
    diff0 = m1-m0
    #switch with frames_per_measure later
    first_beatlen = diff0['eval_measure']* tempo
    #What multiple is needed to normalize tempo for all parts
    tempo_multiple = diff0/first_beatlen
    #normalize tempo by matching length of first note
    df[instrument_lst]=(df / tempo_multiple)[instrument_lst]
    norm_df = df.copy()
    #print("df normalized\n", df)
    #calculate differences between relative note beats
    #access i-2 th element for 
    df_diff = df.diff().dropna().reset_index(drop=True)

    x1 = df_diff.loc[1:n_b-2].reset_index(drop=True)

    x2 = df_diff.loc[0:n_b-3]      
    #ratio between length of adjacent notes 
    ratios = x1.divide(x2) 
    #return ratios
    #print('ratios are\n', ratios)
    #Loop to update
    i = 2

    while i < n_b:
        
        print("\n***ON NOTE: ", i)

        #prev values are all updated
        prev_del = df.loc[i-1] - df.loc[i-2]
        prev_beatlen = prev_del['eval_measure']
        prev_diffs = prev_del[instrument_lst]
        
        #df.loc[i] is not yet updated
        cur_del = df.loc[i] - df.loc[i-1]
        cur_beatlen = cur_del['eval_measure']

        '''
        calculate average predicted location based on other parts
        '''
        prev_rates = prev_diffs/prev_beatlen
        avg_prev_location = np.mean(df[instrument_lst].loc[i-1])

        avg_trajectory_loc = (cur_beatlen *sum(prev_rates)/(n_p)  
                    +  avg_prev_location)
        
        print("\ntrajectory considering only group:", avg_trajectory_loc )

        '''
        location of next note based on original part's ratio
        '''
        #print('previous location should have updated\n', df[instrument_lst].loc[i-1])
        self_trajectory_loc = (df[instrument_lst].loc[i-1]+
                ratios[instrument_lst].loc[i-2]*prev_diffs )
        #print('ratio is \n',  ratios[instrument_lst].loc[i-2])
        #print("prev diffs are\n", prev_diffs)
        print("\ntrajectory considering only self:\n", self_trajectory_loc )
        
        
        
        '''
        perform update
        '''
        
        #print("ABOUT TO UPDATE\n")
        #print("weighted self loc\n",self_weight.loc[i] * self_trajectory_loc )
        #print("weighted avg loc\n",avg_weight.loc[i] * avg_trajectory_loc)
        df.loc[i][instrument_lst] = (self_weight.loc[i] * self_trajectory_loc 
                                     + avg_weight.loc[i] * avg_trajectory_loc) 

        print('\nfinal trajectory:\n',df.loc[i])
        print("\nOriginal location\n", norm_df.loc[i])
        '''
        Ensure all change is positive
        '''
        negatives = df.loc[i]<df.loc[i-1]
        #print("negatives?", sum(negatives))
        if catch_reverse:

            df.loc[i][negatives]= df.loc[i-1]#ensure no negatives 


        i +=1
    return df

# testing

- If a part starts slow, gets faster, then gets slower, the total time will exceed the average time

In [ ]:
#normal, fast then slow 
x=np.array([[1., 2., 3., 4., 5.],
           [0, 1, 1.5, 2, 4],
           [1., 2.1, 3.3, 4.4, 5.5]])
measure = np.array([1,1.25, 1.5, 1.75, 2. ])
columns = ['t1', 't2', 't3']

test_df = pd.DataFrame(x.T, columns = columns)


test_df['eval_measure']= measure
test_df = test_df - test_df.loc[0]


#sim parameters 
time_per_measure = 4
n_b=len(test_df)
n_p = 3

self_weight = test_df[columns]*0+0.5
avg_weight = test_df[columns]*0+0.5
other_instrument_df = {'t1':['t2', 't3'],
                     't2':['t1', 't3'],
                      't3':['t1', 't2']}
instrument_lst = columns

#run sim
new_df = simulate_performance(test_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, 
                         time_per_measure,False)
new_df

In [ ]:
normalize_df(test_df, time_per_measure)

### Plot tempos

In [ ]:
def plot_rates(input_df):
    diff_df = input_df.diff()
    plt.figure()

    for instr in instrument_lst:
        plt.title("Tempo fluctuation: "+instr)
        plt.plot((diff_df['eval_measure']/diff_df[instr]).iloc[:-1], alpha=.5,label=instr)
        plt.legend()    
        plt.show()

# Variable listening version

In [ ]:
#takes only 1 set of listened instruments
def simulate_performance_flex(input_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=False, tempo_correction=False,
                             const=2): #const is reverse addition

    df = input_df.copy()
    #initialize
    m0 = df.loc[0]
    m1 = df.loc[1]
    diff0 = m1-m0
    #switch with frames_per_measure later
    first_beatlen = diff0['eval_measure']* tempo
    #What multiple is needed to normalize tempo for all parts
    tempo_multiple = diff0/first_beatlen
    #normalize tempo by matching length of first note
    df[instrument_lst]=(df / tempo_multiple)[instrument_lst]
    norm_df = df.copy()
    #print("df normalized\n", df)
    #calculate differences between relative note beats
    #access i-2 th element for 
    df_diff = df.diff().dropna().reset_index(drop=True)

    x1 = df_diff.loc[1:n_b-2].reset_index(drop=True)

    x2 = df_diff.loc[0:n_b-3]      
    #ratio between length of adjacent notes 
    ratios = x1.divide(x2) 
    #return ratios
    #print('ratios are\n', ratios)
    #Loop to update
    i = 2

    while i < n_b:
        
        print("\n***ON NOTE: ", i)

        #prev values are all updated
        prev_del = df.loc[i-1] - df.loc[i-2]
        prev_beatlen = prev_del['eval_measure']
        prev_diffs = prev_del[instrument_lst]
        
        #df.loc[i] is not yet updated
        cur_del = df.loc[i] - df.loc[i-1]
        cur_beatlen = cur_del['eval_measure']
        
        '''
        location of next note based on original part's ratio
        '''
        #print('previous location should have updated\n', df[instrument_lst].loc[i-1])
        self_trajectory_loc = (df[instrument_lst].loc[i-1]+
                ratios[instrument_lst].loc[i-2]*prev_diffs )
        #print('ratio is \n',  ratios[instrument_lst].loc[i-2])
        #print("prev diffs are\n", prev_diffs)
        print("\ntrajectory considering only self:\n", self_trajectory_loc )
        
        

        '''
        calculate average predicted location based on other parts
        '''
        prev_rates = prev_diffs/prev_beatlen
        avg_trajectory_loc= prev_rates.copy()#just to initalize, do this better
        #who is listening to who?
        for instr in listens_dct:
            listen_instrs = listens_dct[instr]
            if len(listen_instrs)>0:
                avg_prev_location = np.mean(df[listen_instrs].loc[i-1])

                listen_avg_trajectory_loc = (cur_beatlen *
                                             sum(prev_rates[listen_instrs])/(len(listen_instrs))  
                            +  avg_prev_location)
                avg_trajectory_loc[instr]=listen_avg_trajectory_loc
            #if not listening, just stay own course
            else:
                avg_trajectory_loc[instr]=self_trajectory_loc[instr]

        
        print("\ntrajectory considering only group:", avg_trajectory_loc )


        
        '''
        Ascertain Tempo-based trajectory
        '''
        if tempo_correction:
            tempo_trajectory_loc = df[instrument_lst].loc[i-1] + tempo * cur_beatlen

            print('tempo_trajectory_loc', tempo_trajectory_loc)
            weighted_tempo = tempo_weight[i]* tempo_trajectory_loc
        else:
            weighted_tempo = 0
            
        
        '''
        Put it together
        '''

        #update
        df.loc[i][instrument_lst] = (self_weight.loc[i] * self_trajectory_loc 
                                     + avg_weight.loc[i] * avg_trajectory_loc
                                     +weighted_tempo) 

        print('\nfinal trajectory:\n',df.loc[i])
        print("\nOriginal location\n", norm_df.loc[i])
        '''
        Ensure all change is positive
        '''
        negatives = df.loc[i]<df.loc[i-1]
        #print("negatives?", sum(negatives))
        if catch_reverse:

            df.loc[i][negatives]= df.loc[i-1]+ const#ensure no negatives 


        i +=1
    return df

## Test 0: Test with just bassoon 1 and clarinet 1
- test performed with original algorithm
- they are listening to eachother, shows slowdown clearly

Why does simulation slow down?

Clarinet 1 takes time, or goes slower than expected. Normally, bassoon 1 would not respond to clarinet 1, but it ends up slowing down in the simulation because they are listening to eachother 50/50. Can see this as early as note 9 in the lim simualtion. People tend to take time at the end of phrases, and between "stay constant" and "ritard", the result will be "slightly slower".

To fix this: We want the "engine" listen selectively to other engine parts. Free lines should be about 50/50, staying with the group but doing their own thing

In [ ]:
instrument_lst = ["clarinet_1", 
                  "bassoon_1"]
lim_df = filled_comb_df[['eval_measure', 'clarinet_1', 'bassoon_1']]
n_b = len(lim_df)
n_p = len(instrument_lst)
#self contribution
self_weight = lim_df*0+.5
avg_weight = 1-self_weight
sim_df = simulate_performance(lim_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst,  tempo, False)

### Test 1: clarinet 1, oboe1 as solo, clarinet2, bassoon1 as acc. Just first 16 bars

If only engine instruments listen to eachother, at .5 there is still a little slowdown - guessing bc they are note perfectly steady. 

At .8 there is speed up. 

If following clarinet1 there is slowdown

In [ ]:
solo_acc_df = filled_comb_df[['eval_measure', 'oboe_1', 
                                   'clarinet_1', 'clarinet_2',
                                   'bassoon_1']]

solo_acc_df = solo_acc_df.loc[solo_acc_df['eval_measure']<17]

instrument_lst=['oboe_1',  'clarinet_1', 'clarinet_2',
                                   'bassoon_1']
n_b=len(solo_acc_df)
#self contribution
self_weight = solo_acc_df*0+.8
avg_weight = 1-self_weight

#define 'listens to'
clarinet_1_listens= ['clarinet_1','clarinet_2', 'bassoon_1']
clarinet_2_listens= ['clarinet_2', 'bassoon_1']
bassoon_1_listens = ['clarinet_2', 'bassoon_1']
oboe_1_listens=['oboe_1','clarinet_2', 'bassoon_1']

listens_dct = {'oboe_1':oboe_1_listens, 
              'clarinet_1':clarinet_1_listens,
              'bassoon_1': bassoon_1_listens,
               'clarinet_2':clarinet_2_listens}

simulate_performance_flex(solo_acc_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=False)

## Experiment 1: listen to "backbeat" instruments

- One 'solid beat' instrument, bassoon 2, ignores all other instruments
- "Accompaniment" instruments listen only to solid beat instrument, this way they are not swayed by soloists
- "Solo" instruments listen to accomapniment instrumnets to stay generally on the beat

- In variation where there is no solid beat, interpretation slows down when assumptions break


- Benefit: high amount of specificity possible
- Drawback: need to choose single solid beat instrument or a combination that does not slow down, because slow downs feed into eachother

In [ ]:
solo_acc_df = filled_comb_df.copy()

#solo_acc_df = solo_acc_df.loc[solo_acc_df['eval_measure']<17]

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_b=len(solo_acc_df)
#self contribution
self_weight = solo_acc_df*0+.5
avg_weight = 1-self_weight

tempo=seconds_to_stft_frames(4)

#define 'listens to'
oboe_1_listens=[  'oboe_2','clarinet_1', 'clarinet_2',  'horn_in_e_1', 'horn_in_e_2',
                'bassoon_1','bassoon_2', ]

clarinet_1_listens= [ 'oboe_1', 'oboe_2', 'clarinet_2',  'horn_in_e_1', 'horn_in_e_2',
                'bassoon_1','bassoon_2', ]

bassoon_1_listens = [ 'oboe_1', 'oboe_2','clarinet_1', 'clarinet_2',   'horn_in_e_2',
                'bassoon_2', ]

horn_1_listens = [ 'oboe_1', 'oboe_2','clarinet_1', 'clarinet_2',  'horn_in_e_1', 'horn_in_e_2',
                'bassoon_1','bassoon_2', ]

oboe_2_listens = [ 'clarinet_2',  
                'bassoon_2', 'horn_in_e_2']

clarinet_2_listens= [ 'oboe_2',  
                'bassoon_2', 'horn_in_e_2']

bassoon_2_listens = []

horn_2_listens = [ 'clarinet_2',  
                'bassoon_2', 'oboe_2']



In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
#takes in listens dct

def simulate_performance_listens(input_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=False, tempo_correction=False,
                             const=2): #const is reverse addition

    df = input_df.copy()
    #initialize
    m0 = df.loc[0]
    m1 = df.loc[1]
    diff0 = m1-m0
    #switch with frames_per_measure later
    first_beatlen = diff0['eval_measure']* tempo
    #What multiple is needed to normalize tempo for all parts
    tempo_multiple = diff0/first_beatlen
    #normalize tempo by matching length of first note
    df[instrument_lst]=(df / tempo_multiple)[instrument_lst]
    norm_df = df.copy()
    #print("df normalized\n", df)
    #calculate differences between relative note beats
    #access i-2 th element for 
    df_diff = df.diff().dropna().reset_index(drop=True)

    x1 = df_diff.loc[1:n_b-2].reset_index(drop=True)

    x2 = df_diff.loc[0:n_b-3]      
    #ratio between length of adjacent notes 
    ratios = x1.divide(x2) 
    #return ratios
    #print('ratios are\n', ratios)
    #Loop to update
    i = 2

    while i < n_b:
        
        print("\n***ON NOTE: ", i)

        #prev values are all updated
        prev_del = df.loc[i-1] - df.loc[i-2]
        prev_beatlen = prev_del['eval_measure']
        prev_diffs = prev_del[instrument_lst]
        
        #df.loc[i] is not yet updated
        cur_del = df.loc[i] - df.loc[i-1]
        cur_beatlen = cur_del['eval_measure']
        
        '''
        location of next note based on original part's ratio
        '''
        #print('previous location should have updated\n', df[instrument_lst].loc[i-1])
        self_trajectory_loc = (df[instrument_lst].loc[i-1]+
                ratios[instrument_lst].loc[i-2]*prev_diffs )
        #print('ratio is \n',  ratios[instrument_lst].loc[i-2])
        #print("prev diffs are\n", prev_diffs)
        print("\ntrajectory considering only self:\n", self_trajectory_loc )
        
        

        '''
        calculate average predicted location based on other parts
        '''
        prev_rates = prev_diffs/prev_beatlen
        avg_trajectory_loc= prev_rates.copy()#just to initalize, do this better
        #who is listening to who?
        for instr in listens_dct:
            indicator_row = (listens_dct[instr].loc[i])[instrument_lst]
            listen_instrs = list(indicator_row.loc[indicator_row>0].axes[0])
            if len(listen_instrs)>0:
                avg_prev_location = np.mean(df[listen_instrs].loc[i-1])

                listen_avg_trajectory_loc = (cur_beatlen *
                                             sum(prev_rates[listen_instrs])/(len(listen_instrs))  
                            +  avg_prev_location)
                avg_trajectory_loc[instr]=listen_avg_trajectory_loc
            #if not listening, just stay own course
            else:
                avg_trajectory_loc[instr]=self_trajectory_loc[instr]

        
        print("\ntrajectory considering only group:\n", avg_trajectory_loc )


        
        '''
        Ascertain Tempo-based trajectory
        '''
        if tempo_correction:
            tempo_trajectory_loc = df[instrument_lst].loc[i-1] + tempo * cur_beatlen

            print('tempo_trajectory_loc\n', tempo_trajectory_loc)
            weighted_tempo = tempo_weight[i]* tempo_trajectory_loc
        else:
            weighted_tempo = 0
            

            
        
        '''
        Put it together
        '''

        #update
        df.loc[i][instrument_lst] = (self_weight.loc[i] * self_trajectory_loc 
                                     + avg_weight.loc[i] * avg_trajectory_loc
                                     +weighted_tempo) 

        print('\nfinal trajectory:\n',df.loc[i])
        print("\nOriginal location\n", norm_df.loc[i])
        '''
        Ensure all change is positive
        '''
        negatives = df.loc[i]<df.loc[i-1]
        #print("negatives?", sum(negatives))
        if catch_reverse:

            df.loc[i][negatives]= df.loc[i-1]+ const#ensure no negatives 


        i +=1
    return df

# Experiment: full hand specification

In [ ]:
solo_weight=.7
acc_weight=.2
rest_weight=.01

In [ ]:
self_weight = filled_comb_df*0+.5

In [ ]:
def assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
          self_weight, solo_weight, acc_weight, rest_weight):
    


    for instr in solo:
        #deal with listens dct
        ((listens_dct[instr][instr]).loc[start_idx:stop_idx+1])=0
        for rest_instr in resting:
            ((listens_dct[instr][rest_instr]).loc[start_idx:stop_idx+1])=0


    for instr in acc:
        ((listens_dct[instr][instr]).loc[start_idx:stop_idx+1])=0    
        for solo_instr in solo:
            ((listens_dct[instr][solo_instr]).loc[start_idx:stop_idx+1])=0 
        for rest_instr in resting:
            ((listens_dct[instr][rest_instr]).loc[start_idx:stop_idx+1])=0 
            
    #deal with weight
    
    for instr in solo:
        ((self_weight[instr]).loc[start_idx:stop_idx+1])=solo_weight
    for instr in acc:
        ((self_weight[instr]).loc[start_idx:stop_idx+1])=acc_weight
    for instr in resting:
        ((self_weight[instr]).loc[start_idx:stop_idx+1])=rest_weight

# define listens dictionary

In [ ]:
listens_dct = {}
for instr in instrument_lst:
    listens_dct[instr]= filled_comb_df.copy()
    listens_dct[instr][instrument_lst]=1

In [ ]:
'''
Measures 1-5--> idx 0->34

'clarinet_1' solo -> listens to acc + engine
['horn_in_e_1', 'horn_in_e_2',
                'bassoon_1','bassoon_2', ] acc-> listen  to clarinet 2 and eachother
['clarinet_2']-> only listens to themselves

['oboe_1', 'oboe_2']-> resting, listens only to cl2


'''

solo = ['clarinet_1']
acc = ['horn_in_e_1', 'horn_in_e_2',
                'bassoon_1','bassoon_2', ]
resting = ['oboe_1', 'oboe_2']

start_idx = 0
stop_idx = 34


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)
'''
Measures 5-8--> idx 25:62

'oboe_1' solo -> listens to acc + engine
['horn_in_e_1', 'horn_in_e_2', 'bassoon_1','bassoon_2', 'clarinet_2'] acc-> listen  to clarinet 2 and eachother


['clarinet_1', 'oboe_2']-> resting, listens only to acc


'''
solo = ['oboe_1']
acc = ['horn_in_e_1', 'horn_in_e_2', 'bassoon_1','bassoon_2', 'clarinet_2']
resting = ['clarinet_1', 'oboe_2']

start_idx = 25
stop_idx = 62


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)
'''
Measures 8-13--> idx 62:104

'horn_in_e_1' solo -> listens to acc + engine

['oboe_1', 'oboe_2',  'bassoon_1','bassoon_2', 'clarinet_2'] acc-> listen  to themselves

['horn_in_e_2']-> resting, listens only to acc


'''
solo = ['horn_in_e_1']
acc = ['oboe_1', 'oboe_2',  'bassoon_1','bassoon_2', 'clarinet_2']
resting = ['horn_in_e_2']

start_idx = 62
stop_idx = 104


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)
'''
Measures 13-14.5--> idx 104:116

'bassoon_1' solo -> listens to acc 

['clarinet_1', 'clarinet_2','bassoon_2', 'horn_in_e_1'] acc-> listen  to clarinet 2 and eachother

'''
solo = ['bassoon_1']
acc = ['clarinet_1', 'clarinet_2','bassoon_2', 'horn_in_e_1']
resting = ['oboe_1', 'oboe_2','horn_in_e_2']

start_idx = 104
stop_idx = 116


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)
'''
Measures 14.5-16--> idx 116:128

'horn_in_e_1' solo -> listens to acc 

['clarinet_1', 'clarinet_2','bassoon_2', 'bassoon_1'] acc-> 

'''
solo = ['horn_in_e_1']
acc = ['clarinet_1', 'clarinet_2','bassoon_2', 'bassoon_1']
resting = ['oboe_1', 'oboe_2','horn_in_e_2']

start_idx = 116
stop_idx = 128


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)
'''
Measures 16-20.5--> idx 128:182

['oboe_1', 'oboe_2', 'clarinet_1', 'clarinet_2'] solo -> listens to acc 

['horn_in_e_1', 'horn_in_e_2','bassoon_2', 'bassoon_1'] acc-> 

'''
solo = ['oboe_1', 'oboe_2', 'clarinet_1', 'clarinet_2']
acc = ['horn_in_e_1', 'horn_in_e_2','bassoon_2', 'bassoon_1']
resting = []

start_idx = 128
stop_idx = 182

assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)
'''
Measures 20.5-23--> idx 182:212

['bassoon_1','horn_in_e_1', 'oboe_1'] soli -> listens to acc and eachother

['bassoon_2', 'horn_in_e_2'] acc-> 

'''
solo = []
acc = ['bassoon_1','horn_in_e_1', 'oboe_1','bassoon_2', 'horn_in_e_2']
resting = ['oboe_2', 'clarinet_1', 'clarinet_2']

start_idx = 182
stop_idx = 212


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)
'''
Measures 23-26.5 ---> idx 212:244

listen equally
'''

solo = []
acc = ['oboe_1','oboe_2','clarinet_1', 'clarinet_2',]
resting = ['bassoon_1', 'bassoon_2','horn_in_e_1', 'horn_in_e_2']

start_idx = 212
stop_idx = 244


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)
'''
Measures 26.5-30.5 ---> idx 244:274

['clarinet_1', 'bassoon_2', 'bassoon_1'] --> acc type
'''
solo = ['clarinet_1','bassoon_2', ]
acc = ['bassoon_1']
resting = ['oboe_1','oboe_2', 'clarinet_2', 'horn_in_e_1', 'horn_in_e_2']

start_idx = 244
stop_idx = 274


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

'''
Measures 30.5-34--> idx 274:301

listen equally
'''

solo = ['oboe_2','clarinet_1', 'clarinet_2',]
acc = ['oboe_1']
resting = ['bassoon_1', 'bassoon_2','horn_in_e_1', 'horn_in_e_2']
start_idx = 274
stop_idx = 301


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

'''
Measures 34-->38 idx 301:310

listen equally
'''

solo = []
acc = ['oboe_1','oboe_2','clarinet_1', 'clarinet_2','bassoon_1', 'bassoon_2',]
resting = ['horn_in_e_1', 'horn_in_e_2']
start_idx = 301
stop_idx = 310


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)


'''
Measures 38-->39 idx 310:317

listen equally
'''

solo = ['clarinet_1']
acc = ['oboe_1','oboe_2','clarinet_2','bassoon_1', 'bassoon_2','horn_in_e_1', 'horn_in_e_2']
resting = []
start_idx = 310
stop_idx = 317


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

'''
Measures 39-->43 idx 317:367

listen equally
'''

solo = ['oboe_1']
acc = ['clarinet_1','clarinet_2','bassoon_1', 'bassoon_2','horn_in_e_1', 'horn_in_e_2']
resting = ['oboe_2']
start_idx = 317
stop_idx = 367


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

'''
Measures 43-->48, 367:396

listen equally
'''

solo = ['oboe_1', 'clarinet_1']
acc = ['clarinet_2','bassoon_1', 'bassoon_2','horn_in_e_1', 'horn_in_e_2']
resting = ['oboe_2']
start_idx = 367
stop_idx = 396


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

'''
Measures 48-55, 396:508


'''

solo = ['clarinet_1' , 'oboe_2','horn_in_e_1',]
acc = ['bassoon_2','horn_in_e_2', 'clarinet_2', 'bassoon_1']
resting = ['oboe_1']
start_idx = 396
stop_idx = 508


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

'''
Measures 55-58, 508:556

'''
solo = ['oboe_1']
acc = ['bassoon_2','horn_in_e_1', 'oboe_2', 'clarinet_1',
       'horn_in_e_2', 'clarinet_2', 'bassoon_1']
resting = []
start_idx = 508
stop_idx = 556


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)


'''
Measures 58-60, 556-570

'''
solo = ['oboe_1', ]
acc = ['horn_in_e_1',  'clarinet_1',
        'clarinet_2', 'bassoon_1']
resting = ['oboe_2','horn_in_e_2','bassoon_2',]
start_idx = 556
stop_idx = 570


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

'''
Measures 50-64, 570-623

'''
solo = ['oboe_1', 'clarinet_1','oboe_2','clarinet_2', ]
acc = ['horn_in_e_1',  'horn_in_e_2','bassoon_2',
        'bassoon_1']
resting = []
start_idx = 570
stop_idx = 623


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

'''
Measures 50-64, 570-623

'''
solo = ['oboe_1', 'clarinet_1','oboe_2','clarinet_2', ]
acc = ['horn_in_e_1',  'horn_in_e_2','bassoon_2',
        'bassoon_1']
resting = []
start_idx = 570
stop_idx = 623


assign(listens_dct, solo, acc, resting, start_idx, stop_idx, 
       self_weight, solo_weight, acc_weight, rest_weight)

In [ ]:
self_weight_og=self_weight.copy()

In [ ]:
pd.set_option('display.max_rows', None)
filled_comb_df

# ********Experiment with per row specified changes

In [ ]:
solo_acc_df = filled_comb_df.copy()

#solo_acc_df = solo_acc_df.loc[solo_acc_df['eval_measure']<17]

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_b=len(solo_acc_df)
#self contribution
#tempo contribution
tempo_percent = .03
tempo_weight = np.ones(n_b)*tempo_percent
#self contribution
#self_weight= filled_comb_df*0+.8
self_weight =self_weight*(1-tempo_percent)
#average contribution
avg_weight = (1-tempo_percent)-self_weight
tempo = 700 #keep tempo same

In [ ]:
self_weight

In [ ]:
sim_df = simulate_performance_listens(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=True, tempo_correction=True,
                             const=2)

In [ ]:
sim_df

In [ ]:
diff = sim_df.diff()
for instr in instrument_lst:
    print("\n", instr)
    print(diff.loc[diff[instr]<=0])

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
stretched_full_audio2=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio2, rate=sr_ensemble) 

In [ ]:
write("whole_audio/mozart_simulation_custom_df.wav", sr_ensemble, stretched_full_audio)

# *****Variable leader Experiment (everyone listens to everyone, but preserves leader roles)

In [ ]:
listens_dct = {}
for instr in instrument_lst:
    listens_dct[instr]= filled_comb_df.copy()
    listens_dct[instr][instrument_lst]=1

In [ ]:
solo_acc_df = filled_comb_df.copy()

#solo_acc_df = solo_acc_df.loc[solo_acc_df['eval_measure']<17]

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_b=len(solo_acc_df)
#self contribution
#tempo contribution
tempo_percent = .01
tempo_weight = np.ones(n_b)*tempo_percent
#self contribution
#self_weight= filled_comb_df*0+.8
self_weight =self_weight_og*(1-tempo_percent)
#average contribution
avg_weight = (1-tempo_percent)-self_weight
tempo = 700 #keep tempo same

In [ ]:
sim_df = simulate_performance_listens(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=True, tempo_correction=True,
                             const=2)

In [ ]:
diff = sim_df.diff()
for instr in instrument_lst:
    print("\n", instr)
    print(diff.loc[diff[instr]<=0])

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

# *** Oboe 1 leads

In [ ]:
listens_dct = {}
for instr in instrument_lst:
    listens_dct[instr]= filled_comb_df.copy()
    listens_dct[instr][instrument_lst]=1

In [ ]:
solo_acc_df = filled_comb_df.copy()

#solo_acc_df = solo_acc_df.loc[solo_acc_df['eval_measure']<17]

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_b=len(solo_acc_df)
#self contribution
#tempo contribution
tempo_percent = .05
tempo_weight = np.ones(n_b)*tempo_percent
#self contribution
self_weight= filled_comb_df*0+.1
self_weight['oboe_1']=.9

#average contribution
avg_weight = (1-tempo_percent)-self_weight
tempo = 700 #keep tempo same

In [ ]:
filled_comb_df

In [ ]:
sim_df=simulate_performance_listens(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=False, tempo_correction=True,
                             const=2)

In [ ]:
sim_df

In [ ]:
diff = sim_df.diff()
for instr in instrument_lst:
    print("\n", instr)
    print(diff.loc[diff[instr]<=0])

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
write("whole_audio/mozart_simulation_noTempo.wav", sr_ensemble, stretched_full_audio)

# ******Medium version: all at .6

In [ ]:
listens_dct = {}
for instr in instrument_lst:
    listens_dct[instr]= filled_comb_df.copy()
    listens_dct[instr][instrument_lst]=1

In [ ]:
solo_acc_df = filled_comb_df.copy()

#solo_acc_df = solo_acc_df.loc[solo_acc_df['eval_measure']<17]

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_b=len(solo_acc_df)
#self contribution
#tempo contribution
tempo_percent = .1
tempo_weight = np.ones(n_b)*tempo_percent
#self contribution
self_weight= filled_comb_df*0+.5

#average contribution
avg_weight = (1-tempo_percent)-self_weight
tempo = 700 #keep tempo same

In [ ]:
sim_df=simulate_performance_listens(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=True, tempo_correction=True,
                             const=2)

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
parts = gen_mix_parts(stretched_parts, nonvar_mix, "./parts_audio/sim_.6/")

# .7 leading with corrections ******

In [ ]:
#26.5-34
start_idx1=244
end_idx1 = 301
#77-84
start_idx2 = 769
end_idx2 =794



In [ ]:
listens_dct = {}
for instr in instrument_lst:
    listens_dct[instr]= filled_comb_df.copy()
    listens_dct[instr][instrument_lst]=1

In [ ]:
solo_acc_df = filled_comb_df.copy()

#solo_acc_df = solo_acc_df.loc[solo_acc_df['eval_measure']<17]

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_p=8
n_b=len(solo_acc_df)
#self contribution
#tempo contribution
tempo_percent = .1
tempo_weight = np.ones(n_b)*tempo_percent
#self contribution
self_weight= filled_comb_df*0+.6
self_weight.iloc[start_idx1:end_idx1]=.4 #correct unravelng parts
self_weight.iloc[start_idx2:end_idx2]=.4 #correct unravelng parts
self_weight.iloc[:-10]=.4
#average contribution
avg_weight = (1-tempo_percent)-self_weight
tempo = 700 #keep tempo same

In [ ]:
sim_df=simulate_performance_listens(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=True, tempo_correction=True,
                             const=2)

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

# *** Incorporating hand parsed leader - listens all

In [ ]:

def gen_weights_leader_manual(hand_parsed_lst, filled_comb_df, instrument_lst, 
                             lead_val, non_lead_val):
    prev_val=0
    solo_weight_df = filled_comb_df.copy()*0+non_lead_val
    
    for i in range(len(hand_parsed_lst)):
        #current block
        leader = hand_parsed_lst[i]
        instr=leader[0]
        lims=leader[1]

        #print('lims[0] is ', lims[0])

        cond = ((filled_comb_df['eval_measure']<=lims[1])&
                        (lims[0] <=filled_comb_df['eval_measure']) )

 
        #print(cond)

        solo_weight_df[instr].loc[cond]= lead_val


    return solo_weight_df

In [ ]:
solo_weights =gen_weights_leader_manual(hand_parsed_leader1, filled_comb_df, instrument_lst, 
                             .8, .1)

In [ ]:
listens_dct = {}
for instr in instrument_lst:
    listens_dct[instr]= filled_comb_df.copy()
    listens_dct[instr][instrument_lst]=1

In [ ]:
solo_acc_df = filled_comb_df.copy()

#solo_acc_df = solo_acc_df.loc[solo_acc_df['eval_measure']<17]

instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_b=len(solo_acc_df)
#self contribution
#tempo contribution
tempo_percent = .2
tempo_weight = np.ones(n_b)*tempo_percent
#self contribution
self_weight= solo_weights

#average contribution
avg_weight = (1-tempo_percent)-self_weight
tempo = 700 #keep tempo same

In [ ]:
sim_df=simulate_performance_listens(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, listens_dct,
                         tempo,catch_reverse=False, tempo_correction=True,
                             const=2)

In [ ]:
diff = sim_df.diff()
for instr in instrument_lst:
    print("\n", instr)
    print(diff.loc[diff[instr]<=0])

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

### Experiment 2: Incorporation of target tempo loss

- each instrument listens to all other instruments
- does not slow down because target tempo is incorporated


In [ ]:
#4 seconds per measure 

n_b=len(filled_comb_df)
instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_p = len(instrument_lst)


In [ ]:
other_instrument_dct

In [ ]:
tempo=seconds_to_stft_frames(3.7)

#tempo contribution
tempo_percent = .15
tempo_weight = np.ones(n_b)*tempo_percent
#self contribution
self_weight= filled_comb_df*0+.9
self_weight *=(1-tempo_percent)
#average contribution
avg_weight = (1-tempo_percent)-self_weight


In [ ]:
self_weight

In [ ]:
sim_df = simulate_performance_flex(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, 
                         instrument_lst, other_instrument_dct,
                         tempo,catch_reverse=False, tempo_correction=True)

In [ ]:
sim_df

In [ ]:
diff = sim_df.diff()
for instr in instrument_lst:
    print("\n", instr)
    print(diff.loc[diff[instr]<=0])

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
write("whole_audio/mozart_simulationTempo.wav", sr_ensemble, stretched_full_audio)

# Experiment 3

- An instrument's 'freedom' can vary over time, where freedom is the importance placed on following their original part vs. following the group

- Posit: When an instrument is resting, it has no freedom. When it is not resting, it has a lot of freedom


### This function "listens" only to notes that are currently playing

- Note: for rests, everyone needs equal wieghtt (can't be row of zeros)

Possibilties

|                             | You are currently playing                 |You are currently resting |
|-----------------------------|-------------------------------------------|----------------------------|
| You were previously playing | s_w x self_trajectory + a_w x avg playing trajectory  |  0 x self_trajectory + 1 x avg playing trajectory |
| You were previously resting | s_w x self_trajectory + a_w x avg playing trajectory|  0 x self_trajectory + 1 x avg playing trajectory |



where avg playing trajectory is who was previously playing and where they would be now

In [ ]:
def simulate_performance_curplay(input_df, n_b, n_p, 
                         self_weight, avg_weight, tempo_weight,
                         instrument_lst, tempo, catch_reverse=True):


    #Note: check self_weight, there cannot be a row of zeros?
    df = input_df.copy()
    '''
    Initialize simuation parameters 
    '''
    m0 = df.loc[0]
    m1 = df.loc[1]
    diff0 = m1-m0
    #switch with frames_per_measure later
    first_beatlen = diff0['eval_measure']* tempo
    #What multiple is needed to normalize tempo for all parts
    tempo_multiple = diff0/first_beatlen
    #normalize tempo by matching length of first note
    df[instrument_lst]=(df / tempo_multiple)[instrument_lst]
    
    #calculate differences between relative note beats
    #access i-2 th element for 
    df_diff = df.diff().dropna().reset_index(drop=True)

    x1 = df_diff.loc[1:n_b-2].reset_index(drop=True)

    x2 = df_diff.loc[0:n_b-3]      
    #ratio between length of adjacent notes 
    ratios = x1.divide(x2)  


    #Loop to update
    i = 2

    while i < n_b:
        print("\n***ON NOTE: ", i)

        #prev values are all updated
        prev_del = df.loc[i-1] - df.loc[i-2]
        prev_beatlen = prev_del['eval_measure']
        prev_diffs = prev_del[instrument_lst]
        
        #df.loc[i] is not yet updated
        cur_del = df.loc[i] - df.loc[i-1]
        cur_beatlen = cur_del['eval_measure']

        

        '''
        Average predicted location based on other parts
        Figure out which parts were previously playing
        - 0 indicates playing
        '''
        
                
        
        prev_rates = prev_diffs/prev_beatlen


        #print("weights of previous note\n", self_weight.iloc[i-1])
        prev_weights = self_weight.iloc[i-1]
        
        #pick out nonzero elements 
        prev_weights = prev_weights.loc[prev_weights !=0]
        #print("prev_weights", prev_weights)
        
        #extract which intruments were previously playing
        prev_playing_instrs = list(prev_weights.keys())
        #print('instruments previously playing', prev_playing_instrs)
        
        #calculate trajectory
        pred_trajectory_loc = (prev_rates[prev_playing_instrs]*cur_beatlen
                             ) + df[prev_playing_instrs].loc[i-1]
        print("current trajectory for all instruments\n", pred_trajectory_loc)


        avg_trajectory_loc = sum( pred_trajectory_loc)/len(prev_playing_instrs)            

        '''
       
        
        #make this a weighted sum: i.e. parts that listen less should contribute less
        prev_instr_avg_weight = avg_weight[ prev_playing_instrs ].loc[i-2]
        prev_instr_avg_weight=prev_instr_avg_weight/sum(prev_instr_avg_weight)
        print('previous instrument weight contribution to average trajectory:\n',
              prev_instr_avg_weight)
        avg_trajectory_loc =sum(prev_instr_avg_weight
                                *pred_trajectory_loc)
        '''       
        print("average trajectory loc is \n", avg_trajectory_loc)

        '''
        Ascertain Each intrument's own part-based trajecotry
        '''
        #location of next note based on original part's ratio
        #look at ratios indexing 
        self_trajectory_loc = df[instrument_lst].loc[i-1]+ prev_diff*ratios.loc[i-2]
        
        print("self_trajectory_loc", self_trajectory_loc)

        '''
        Ascertain Tempo-based trajectory
        '''
        
        tempo_trajectory_loc = df[instrument_lst].loc[i-1] + tempo * cur_beatlen

        print('tempo_trajectory_loc', tempo_trajectory_loc)
        
        '''
        Put it together
        '''

        #update
        df.loc[i][instrument_lst] = (self_weight.loc[i] * self_trajectory_loc 
                                     + avg_weight.loc[i] * avg_trajectory_loc
                                     +tempo_weight[i]* tempo_trajectory_loc) 

        #print(df)
        
        '''
        Ensure all change is positive
        '''
        negatives = df.loc[i]<df.loc[i-1]
        print("negatives?", sum(negatives))
        if catch_reverse:

            df.loc[i][negatives]= df.loc[i-1]+ 2#ensure no negatives 
            

        i +=1
    return df

### Define play indicator

Play indicator has 1 when beat is represented by note onset, 0 otherwise

In [ ]:
play_indicator = comb_df.fillna(0)
play_indicator[play_indicator>0]=1
play_indicator['eval_measure']= comb_df['eval_measure']
play_indicator

## Example 1 
- Everyone listens 50% and follows 50%
- Everyone "listens" only parts that are currently playing
- Target tempo is 3.7 seconds per measure, and that has a 10% contribution to descision

In [ ]:
#4 seconds per measure 

n_b=len(filled_comb_df)
instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_p = len(instrument_lst)


In [ ]:
tempo=seconds_to_stft_frames(3.7)

#tempo contribution
tempo_percent = 0
tempo_weight = np.ones(n_b)*tempo_percent
#self contribution
self_weight = play_indicator[instrument_lst].copy()
self_weight= self_weight*.4
self_weight['oboe_1']= .8
self_weight *=(1-tempo_percent)
#average contribution
avg_weight = (1-tempo_percent)-self_weight


In [ ]:
self_weight

In [ ]:
sim_df = simulate_performance_curplay(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, tempo_weight,
                         instrument_lst, tempo)

In [ ]:
sim_df

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)


In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
#write("50-50.wav", sr_ensemble, stretched_full_audio)

## Graph spread of ensemble error compared to average of each time point

In [ ]:
final_time_arr = sim_df[instrument_lst].to_numpy()

avg_times = np.array([np.sum(final_time_arr, axis=1)/num_voices]).T

all_difs = final_time_arr - avg_times

plt.title("Simulation: 50% listen and 50% lead, 20% tempo")
plt.hist(all_difs.flatten(), bins=100)
plt.xlabel("Difference from mean times (in frames)")
plt.ylabel("Number of timepoints")
plt.show()
print("Inter quartile range is", iqr(all_difs.flatten()))
print("Maximum deviation", np.max(np.abs(all_difs)))

### Example 3 Hand-parsed ('best possible version')

In [ ]:
def make_hand_parsed_weights_mult(comb_df, play_indicator, 
                             instrument_lst, hand_parsed_lst, 
                             lead_weight, non_lead_weight):

    self_weights = play_indicator.copy()
    #
    self_weights[self_weights==0.01]=0
    #self_weights  = self_weights * non_lead_weight
    print(self_weights)
    for el in hand_parsed_lst:
        instr = el[0]
        lims=el[1]
        #print(instr)

        #print(lims)
        cond = ((lims[0] <= self_weights['eval_measure'] ) 
                & (self_weights['eval_measure']<lims[1]))
        #print(cond)
        self_weights[instr][cond]=self_weights[instr][cond]*1.5#lead_weight
        #print(self_weights)
    return self_weights  
        

In [ ]:
def make_hand_parsed_weights(comb_df, 
                             instrument_lst, hand_parsed_lst, 
                             lead_weight, non_lead_weight):

    self_weights = comb_df.fillna(0)
    self_weights[self_weights>0]=1
    self_weights['eval_measure']= comb_df['eval_measure']

    self_weights  = self_weights * non_lead_weight
    print(self_weights)
    for el in hand_parsed_lst:
        instr = el[0]
        lims=el[1]
        #print(instr)

        #print(lims)
        cond = ((lims[0] <= self_weights['eval_measure'] ) 
                & (self_weights['eval_measure']<lims[1]))
        #print(cond)
        self_weights[instr][cond]=lead_weight
        #print(self_weights)
    return self_weights  
        

In [ ]:
#tend to follow more if more people playing. 
variable_weights = pd.DataFrame(make_weight_arr(comb_df,instrument_lst, 0.01 ), columns=instrument_lst)
variable_weights[variable_weights<=0.01]=0
variable_weights['eval_measure']= filled_comb_df['eval_measure']

In [ ]:
variable_weights

In [ ]:
make_hand_parsed_weights(comb_df, variable_weights, 
                             instrument_lst, hand_parsed_leader1, 
                             .8, .1)

In [ ]:
#4 seconds per measure 

n_b=len(filled_comb_df)
instrument_lst = ["oboe_1", 
                  "oboe_2", 
                  "clarinet_1", 
                  "clarinet_2", 
                  "bassoon_1", 
                  "bassoon_2", 
                  "horn_in_e_1", 
                  "horn_in_e_2"]
n_p = len(instrument_lst)


In [ ]:
tempo=seconds_to_stft_frames(4)

#tempo contribution
tempo_percent = 0
tempo_weight = np.ones(n_b)*tempo_percent

self_weight = make_hand_parsed_weights(comb_df, 
                             instrument_lst, hand_parsed_leader1, 
                             .8, .2)[instrument_lst]


self_weight *=(1-tempo_percent)

avg_weight = (1-tempo_percent)-self_weight

In [ ]:
sim_df = simulate_performance_curplay(filled_comb_df, n_b, n_p, 
                         self_weight, avg_weight, tempo_weight,
                         instrument_lst, tempo)

In [ ]:
sim_df

In [ ]:
final_stretch = assign_new_stft(df_dct, sim_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)

In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

# Partition

1. set tolerance

2. Loop:
    - a. Start new section
    - b. Increment section by 1 timepoint
    - c. scale beginning and end to match
    - d. check if tolerance exceeded
    - e. If tolerance exceeded, rollback 1 timepoint and start new section (a)
    - f. If tolerance not exceeded, continue incrementing (b)


In [ ]:
def scale_and_check_error(partition_df_ref,
                          indicator_df, start_idx, stop_idx, tol):
    section = partition_df_ref.iloc[start_idx:stop_idx+1] #get new version memory hog

    #number of measures the section covers 
    dist_elapsed = (section['eval_measure'].loc[stop_idx]-
                section['eval_measure'].loc[start_idx])
    #not used, should be this much time elapsed for section
    time_elapsed = dist_elapsed * tempo
    #scale the section - assume first value is equivalent for all
    tempo_multiplier = dist_elapsed/(
        section.loc[stop_idx]- section.loc[start_idx])   
    

    # assign in new dataframe (will change every iteration)
    scaled_section = tempo*(section[instrument_lst]* (tempo_multiplier[instrument_lst]))
    #rest start to 0
    scaled_section[instrument_lst]-=scaled_section[instrument_lst].iloc[0]
    #Check if error has exceeded 
    error_exceeded = False
    print("***************************")
    for i in range(start_idx, stop_idx+1):
        scaled_section_row = scaled_section.loc[i]
        indicator_section_row= indicator_df.loc[i]
        curplay_instrument_series = scaled_section_row.loc[indicator_section_row>0]
        mean = np.mean(curplay_instrument_series)
        square_error =(curplay_instrument_series-mean)**2
        print("\nsquare error", square_error)
        #One instrument is too far from mean location
        if sum(square_error>tol)>0:
            error_exceeded=True
            break
    print("**************************")      
    
    return error_exceeded, scaled_section


## Experiment 1: absolute toleralance of 25 frames
- single tolerance applied for all notes
- tolerance could change depending on application (e.g. array of tolerances depending details of score)
- this is a bit too much error for my taste

In [ ]:
stft_frames_to_seconds(8) #slighly more than a 32nd note at quarter = 60

In [ ]:
seconds_to_stft_frames(.0625)

In [ ]:

partition_df_ref = filled_comb_df.copy()
tol_abs=15
tempo=seconds_to_stft_frames(4.1)
n_b = len(partition_df_ref )

In [ ]:
#Input: 
#filled comb df (with interpolation)
#comb_df (with NaN values in rests)
#tolerance: absolute number of frames a timepoint can differ from the mean of that beat
def create_partition_df_pointwise(partition_df_ref, comb_df, tol_abs, n_b, tempo):
    
    tol=tol_abs**2
    partition_df_new=filled_comb_df.copy()
    indicator_df = comb_df.fillna(0)
    
    start_idx=0
    section_borders_lst = []
    while start_idx < n_b-1:
        print('\n********\nnew section starting at',start_idx)
        stop_idx = start_idx
        section_borders_lst.append(partition_df_ref['eval_measure'].iloc[start_idx])
        #stop_idx can theoretically increase until the end of the piece
        while stop_idx < n_b-1:
            #increment 
            stop_idx +=1
            print('\nsection stopping at', stop_idx)
            #get scaled version
            isExceeded, scaled_section = scale_and_check_error(partition_df_ref,
                                                               indicator_df, 
                                                               start_idx, stop_idx, tol)

            #a False isExceeded means we repeat the process
            if isExceeded == True:
                
                print("!!! tolerance exceeded ")
                #roll back
                stop_idx -=1
             #get scaled version
                isExceeded, scaled_section = scale_and_check_error(
                    partition_df_ref,indicator_df, start_idx, stop_idx, tol)

                shifted_scaled_section = (scaled_section[instrument_lst]+ 
                    partition_df_new[instrument_lst].iloc[start_idx])
                print("final scaled section is\n", shifted_scaled_section)
                #assign 
                '''
                print('putting in\n', scaled_section[instrument_lst]+ 
                    partition_df_new[instrument_lst].iloc[start_idx])
                print('assigning to\n', partition_df_new[instrument_lst].iloc[start_idx:stop_idx+1])
                '''
                (partition_df_new.iloc[start_idx:stop_idx+1])[instrument_lst]= (
                    scaled_section[instrument_lst]+ 
                    partition_df_new[instrument_lst].iloc[start_idx])

                print('new times\n',partition_df_new[instrument_lst].iloc[start_idx:stop_idx+1])

                break
            #Must also stop if we've reached the end of the piece
            elif stop_idx == n_b-1:
                print("Need to end, currently stop is", n_b-1)
                shifted_scaled_section = (scaled_section[instrument_lst]+ 
                    partition_df_new[instrument_lst].iloc[start_idx])
                print("final scaled section is\n", shifted_scaled_section)
                #assign 
                (partition_df_new.iloc[start_idx:stop_idx+1])[instrument_lst]= (
                    scaled_section[instrument_lst]+ 
                    partition_df_new[instrument_lst].iloc[start_idx])

            
        start_idx = stop_idx
        print(start_idx, stop_idx)
    return partition_df_new, section_borders_lst

In [ ]:
partition_df, section_lst=create_partition_df_pointwise(filled_comb_df, comb_df, 9, n_b, tempo)

In [ ]:
partition_df

In [ ]:
len(section_lst)

In [ ]:
final_stretch = assign_new_stft(df_dct, partition_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)

In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
write("whole_audio/mozart_partition_tolerance_9.wav", sr_ensemble, stretched_full_audio)

## Experiment 2: Ensure strong beat alignment
1. set tolerance

2. Loop:
    - a. Start new section.
        - a1. Availible increments depend on modulus of measure. e.g. 2.25 could increment by quarter, while 2.5 could increment by half note or quarter. Increment must be in set [.0625, .125, .25, .5, 1, 2, 4, 8]
    - b. Increment section by 1 timepoint
    - c. scale beginning and end to match
    - d. check if tolerance exceeded
    - e. If tolerance exceeded, rollback 1 timepoint and start new section (a)
    - f. If tolerance not exceeded, continue incrementing (b)


In [ ]:
filled_comb_df.loc[filled_comb_df['eval_measure']==20.1].index[0]

In [ ]:
increment_vals = [.0625, .125, .25, .5, 1., 2., 4., 8., 16.]

In [ ]:
#Input: 
#filled comb df (with interpolation)
#comb_df (with NaN values in rests)
#tolerance: absolute number of frames a timepoint can differ from the mean of that beat

# while start < end: 
#start with default size of 2. 
#if error not exceeded, increment, which will break out of loop
#else, divdie size by 2

def create_partition_df_bounded(partition_df_ref, 
                                comb_df, tol_abs, n_b, 
                                tempo, default_measure=2):
    
    tol=tol_abs**2
    partition_df_new=filled_comb_df.copy()
    indicator_df = comb_df.fillna(0)
    
    start_idx=0
    prev_idx = 0
    section_borders_lst = []
    while start_idx < n_b-1:

        start_measure = partition_df_ref['eval_measure'].iloc[start_idx]        
        section_stop_measure = start_measure+ default_measure #set max distance   
        print("SECTION MUST STOP AT ", section_stop_measure)
        test_length = default_measure
        #termination test:
        while start_measure < section_stop_measure:     
      
            try: 

                #this is stopping point, will throw index error if does not exist
                stop_idx = partition_df_ref.loc[partition_df_ref['eval_measure']==start_measure+test_length].index[0]
                print("------")
                print("starting at index", start_idx,"= measure", start_measure)
                stop_measure = partition_df_ref['eval_measure'].iloc[stop_idx]
                print("trying a stopping index of ", stop_idx, '= measure', stop_measure)
                #test error level
                isExceeded, scaled_section = scale_and_check_error(
                        partition_df_ref,indicator_df, start_idx, stop_idx, tol)

                #scale section if error not exceeded
                if isExceeded == False:
                    shifted_scaled_section = (scaled_section[instrument_lst]+ 
                        partition_df_new[instrument_lst].iloc[start_idx])
                    print("final scaled section is\n", shifted_scaled_section)
                    #assign 
                    (partition_df_new.iloc[start_idx:stop_idx+1])[instrument_lst]= (
                        scaled_section[instrument_lst]+ 
                        partition_df_new[instrument_lst].iloc[start_idx])
                    #record old section beginning location
                    section_borders_lst.append(start_measure)
                    #increment
                    start_idx = stop_idx
                    start_measure = partition_df_ref['eval_measure'].iloc[start_idx]  
                else:
                    #deal with this in exception clause
                    raise ValueError("error exceeded at length "+str(test_length))

            
            except Exception as e:
                print(e)
                #ending clause
                if start_idx ==n_b-1:
                    break
                
                test_length /= 2
                print("test length halved is ", test_length)
                #check if this stop measure will work
                min_ending_measure = partition_base_df['eval_measure'].iloc[start_idx+1]
                min_length = min_ending_measure - start_measure
                print("next stop point in length", min_length)
                if min_length > test_length:
                    test_length = min_length
        
                print("final test length is ", test_length)
    return partition_df_new, section_borders_lst

In [ ]:
16 % 1

In [ ]:
partition_base_df=filled_comb_df.copy()
partition_base_df['eval_measure']= filled_comb_df['eval_measure']-1
partition_base_df

In [ ]:
partition_df, section_lst=create_partition_df_bounded(partition_base_df, 
                                                      comb_df, 500, n_b, tempo, 
                                                      default_measure = 1)

In [ ]:
pd.set_option('display.max_rows', None)
partition_df

In [ ]:
section_lst

In [ ]:
final_stretch = assign_new_stft(df_dct, partition_df, instrument_lst)
stretched_parts = create_final_parts(ensemble_audio_lst,final_stretch,instrument_lst, 2)

In [ ]:
parts = gen_mix_parts(stretched_parts, nonvar_mix, "./parts_audio/big_beat_1meas/")

In [ ]:
stretched_full_audio=sum_parts(stretched_parts)
ipd.Audio(stretched_full_audio, rate=sr_ensemble) 

In [ ]:
section_lst